# Lecture 18: Desiigner's Favorite Lecture

### Please note: This lecture will be recorded and made available for viewing online. If you do not wish to be recorded, please adjust your camera settings accordingly. 

# Reminders/Announcements:
- Assignment 6 due tomorrow at 8pm.
- Final Project Group/Topic Assignments are available.
- Quiz 2 is on February 22nd. Please see the Canvas announcement for details.
    - Lecture on that Monday will just be cancelled. Twenty minutes will not get us very far with any topic these days...

## Pandas

![](Panda-2.jpg)

From the pandas website: "pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language." The name comes from the phrase *PANel DAta*. The main purpose of pandas is to read, manipulate, store, and prepare datasets.

Note! We are in the Python kernel again!

For good introductions to Pandas, I recommend:
- 10 minutes to pandas (although this takes *much longer* than 10 minutes...): https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html 
- Python for Data Analysis (a book by the main developer of pandas): https://www.oreilly.com/library/view/python-for-data/9781449323592/
- Hands on Machine Learning with Scikit-Learn, Keras, & Tensorflow (a machine learning focused treatment): https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/

I will do my best to give a workable discussion today, but really there is a lot to explore with pandas...

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

The main concepts in pandas are *Series* and *DataFrames*. These are similar to 1D and 2D arrays in NumPy. We will focus on the DataFrames. A DataFrame is like a mix between a 2D numpy array and an excel file. You can make DataFrames by hand

In [0]:
pd.DataFrame([[1,2],[3,4]])

but a common way of making them is to feed in a csv or excel file. Let's explore this with the help of some old friends of mine:

In [0]:
df = pd.read_csv('pokemon.csv', index_col = 1)   #Reads in a comma separated value file and turns it into a dataframe.
print(type(df))
print('**************************************************')
print(df)

It formats much better if you forget the print!

In [0]:
df

Note that the rows are indexed by the pokemon name; this was decided by the optional delimiter in the read command.

As mentioned last week, the first thing you should do with data is some *cursory analysis*.

In [0]:
df.columns #Labels of the columns

In [0]:
df.index  #Labels of the rows (sometimes these will just be integers...)

In [0]:
df.head(7) #First few rows

In [0]:
df.tail(10) #Last few rows

In [0]:
df.sample(n = 8) #Random collection of rows

In [0]:
df.sample(frac = 1/100)  #Random collection of rows

In [0]:
df.info() #Summarize the DataFrame

In [0]:
df.describe()  #Summarize the DataFrame in a different way

In [0]:
df.hist(bins = 20)
plt.show()

What can we see from this basic summary? Quite a bit if we look! For example:
- pokemon seem to come in "weak, average, or strong" forms (the "total" plot seems trimodal)
- some stats are "more spread out" than others (compare Sp. Atk to Sp. Def)

Let's look a bit deeper! Let's look for *correlations* between the data. First, let's extract the *numerical skill attributes*:

In [0]:
attr = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
skills = df[attr]
skills.head()

In [0]:
from pandas.plotting import scatter_matrix
scatter_matrix(skills, figsize = (15,15))
plt.show()

Can we quantify this?

In [0]:
skills.corr()

## A note on correlation coefficients...

The above matrix is computing *standard correlation coefficients* of each pair (also called *Pearson's r value*). It computes how strong of a *linear* relationship exists between the variables. It can be helpful, but you have to be careful. See this picture from Wikipedia (citation:  By DenisBoigelot, original uploader was Imagecreator - Own work, original uploader was Imagecreator, CC0, https://commons.wikimedia.org/w/index.php?curid=15165296 )

![](r.png)

## Back to the dataframe

How do you work with these data frames?

A key tool is *loc* and *iloc*. The *loc* command extracts a row by its *named index*. The *iloc* command extracts a row by an *integer index* (be careful with this data set! The hashtag column is different than the row's index...)

In [0]:
df.head()

In [0]:
df.loc['Bulbasaur']

In [0]:
df.iloc[0]

You can use multiindexing (like with numpy) to simultaneously restrict the columns:

In [0]:
df.loc['Bulbasaur','Speed']

In [0]:
df.loc['Bulbasaur',['HP','Speed','Generation']]

If you don't want to bother writing the names, that is the perfect time to use iloc:

In [0]:
df.iloc[:8,:5]

In [0]:
df.iloc[:8:2,[1,2,3,7]]

A *very* powerful tool is to use *masking* with the loc function like so:

In [0]:
df.loc[df['Attack'] > 170]

How is this working? The inner argument gives us a Boolean condition for each row:

In [0]:
bools = df['Attack'] > 170
bools

Pandas then looks for the rows where the bool is *True*

In [0]:
df.loc[bools]

You can string these together, if you get the syntax right:

In [0]:
df.loc[((df['Attack'] > 170) and df['Legendary'])]

Instead of using the Python commands you're used too, you'll have to use 
- & for "and"
- | for "or"
- ~ for "not"

In [0]:
df.loc[((df['Attack'] > 170) & (df['Legendary']))]

In [0]:
df.loc[((df['Attack'] > 170) | (df['HP'] >170))]

## Adding Data

We can also do entrywise operations, like with numpy, and we can use this to add data to our frame:

In [0]:
df['Offensive'] = df['Speed'] + df['Attack'] + df['Sp. Atk']
df['Defensive'] = df['HP'] + df['Defense'] + df['Sp. Def']
df.head()

Hmmm... this seems nice, but maybe we could rearrange it?

In [0]:
df.iloc[:,[0,1,2,-4,-3,3,5,7,9,-2,4,6,8,-1]]

In [0]:
newdf = df.iloc[:,[0,1,2,-4,-3,3,5,7,9,-2,4,6,8,-1]]
newdf.head()

You can also *change* entries in the dataframe in many ways:

In [0]:
copy = newdf.copy() #Copy the data so we don't mash up our original dataset...
copy.head()

The simplest method is to use loc to find the index you want to change, and then simply reassign that value:

In [0]:
copy.loc['Bulbasaur', 'Generation'] = 5
copy.head()

Once you get more comfortable with the loc command, you can do pretty crazy stuff:

In [0]:
copy.loc[copy['Type 1'] == 'Grass', 'Type 1'] = 'Vegetable'
copy.head(7)

How did this work? The mask `copy['Type 1'] == 'Grass'` finds the pokemon with Type 1 given by Grass. The second parameter `Type 1` then gives the item we are going to change for these pokemon.

If you were balancing the game, you could use this to uniformly "buff" or "debuff" certain characters:

In [0]:
copy.loc[copy['Type 1'] == 'Fire', 'Defense'] = copy['Defense'] + 10
copy.head(7)

Be careful! You would also want to update other columns in your dataframe if you did this...

## ************* Participation Check***********************
Take the `copy` dataframe and modify the 'Total' and 'Defensive' columns to reflect the "buff" for fire pokemon's defense (there are *many* ways to do this). Once you have done your transformation, call `copy.head(7)` to make sure you accomplished what you want.

## **********************************************************

## Sorting

Now that you have the data, what would you do to make the best pokemon team? You'd probably want to sort the skills somehow...

In [0]:
newdf.sort_values('Total').head()

Whoops! This gave me the worst pokemon...

In [0]:
newdf.sort_values('Total', ascending = False).head()

You can also sort alphabetic data, and use multiple keys at the same time:

In [0]:
newdf.sort_values('Type 1').head()

In [0]:
newdf.sort_values(['Type 1','Total'],ascending = [True, False]).head()

Ok, what if you *really need a defensive fire type*, but you aren't allowed to use legendary pokemon...

## *********** Participation Check ******************************************
From `newdf` extract a dataframe `fire` consisting of the pokemon which are a *fire type* (either Type 1 equals Fire or Type 2 equals Fire) and which are *not legendary*. Sort the resulting dataframe to find the "best defensive" choice.

## ************************************************************************************

Finally, you can sort by *index* using `sort_index`. As always, this is not always useful, but occasionally you may want to!

In [0]:
newdf.sort_index(axis = 0).head() # Sort by ROW index

In [0]:
newdf.sort_index(axis = 1).head() # Sort by COLUMN index

## The groupby Command

You can aggregate your data quite easily in pandas using the groupby command:

In [0]:
newdf.groupby(['Type 1']).mean()

Additionally, we can sort!

In [0]:
newdf.groupby(['Type 1']).mean().sort_values('Defense',ascending = False)

Additional commands are *sum* and *count*. These don't always make sense, but occasionally they're useful!

In [0]:
newdf.groupby(['Type 1']).sum().sort_values('Defense',ascending = False)

In [0]:
newdf.groupby(['Type 1']).count().sort_values('Defense',ascending = False)

## *********** Participation Check ******************************************
Try passing *a list* of parameters into `groupby`, instead of just the string 'Type 1'. What happens then? Which pair of types `(Type 1, Type 2)` has the worst average speed?

## *******************************************************************

## Saving

What good would all this analysis be if we couldn't save our results? Thankfully this is very easy!

In [0]:
newdf.head()

In [0]:
newdf.to_csv('ModifiedPokemon.csv')

In [0]:
newdf.to_excel('ModifiedPokemon.xlsx')

## Pandas and Regression (Time Permitting)

Just like numpy arrays, you can use pandas dataframes as inputs into different models, such as a linear regression. Let's examine housing prices in California. This data is taken from the 1990 US Census, and displays data on (essentially) neighborhood level data. Data obtained from Aurelien Geron (https://github.com/ageron) who obtained it from Luis Torgo (https://www.dcc.fc.up.pt/~ltorgo/Regression/)

In [0]:
housing = pd.read_csv('housing.csv')
housing.head()

In [0]:
housing.describe()

In [0]:
housing.info()

Uh oh! We are missing some values in the "total_bedrooms" column. Let's *impute* by replacing 'null entries' with the corresponding mean. In pandas, blank entries are usually represented by `NaN`s. You can test for this by using the `isna()` command:

In [0]:
housing.loc[housing['total_bedrooms'].isna()].head()

Using our knowledge from above, we can now "fill in the blanks"

In [0]:
housing.loc[housing['total_bedrooms'].isna(), 'total_bedrooms'] = 537.870553

In [0]:
housing.info()

Great! Let's keep going. Do you think this really represents California housing?

In [0]:
housing.plot(kind = 'scatter', x = 'longitude', y = 'latitude')
plt.show()

Remember that `alpha` parameter from way back long ago that seemed useless? It is *very* useful to describe *densities* of geographic data!

In [0]:
housing.plot(kind = 'scatter', x = 'longitude', y = 'latitude', alpha = .1)
plt.show()

What about those color map things?

In [0]:
housing.plot(kind = 'scatter', x = 'longitude', y = 'latitude', alpha = .4, s = housing['population']/100, label = 'population',c = 'median_house_value', cmap = 'jet')
plt.show()

Lets try to use this dataset to predict housing prices. But first; does it really make sense to consider the "total rooms" in the district? Is there a better option?

Also, do we really want total rooms *and* total bedrooms? Those are probably highly correlated...maybe we can just keep the *ratio* of rooms to bedrooms:

Maybe this will be better...lets try!

In [0]:
from sklearn.linear_model import LinearRegression
ols = LinearRegression()

In [0]:
ols

In [0]:
ols.fit(housing[['housing_median_age','population','median_income', 'roomsPerHouse','bedroomsPerRoom']], housing['median_house_value'])

In [0]:
ols.coef_

In [0]:
ols.intercept_

Looking good! What if we wanted to handle the categorical attribute? (Proximity to the ocean?)

## Categorical Attributes and "Pipelines"

Scikit learn offers many preprocessing tools which can be used to form *pipelines* on your dataset. This is essentially automated cleaning and processing of your data; for instance, when we *imputed* missing values above, that was a common "pipeline" technique.

In [0]:
proximity = housing[['ocean_proximity']]
proximity.head(10)

As with the "Boy/Girl" scenario in Lecture 17, we want to translate this to numeric data somehow. Scikit Learn has several builtin encoders for categorical data. The first method simply labels the data with an integer:

In [0]:
from sklearn.preprocessing import OrdinalEncoder
ordEncode = OrdinalEncoder()
ordEncode.fit_transform(proximity)

In [0]:
ordEncode.categories_

In [0]:
pOrd = proximity.copy()
pOrd['Ord'] = ordEncode.fit_transform(proximity)
pOrd.head()

In [0]:
pOrd.sample(n=10)

This *can be fine* in some cases, such as in a survey: if you had the choices very bad, bad, average, good, very good, then you could reasonably relabel this as 0,1,2,3,4. In this case it does not make as much sense, so we can instead use a "one hot" encoding. This creates *five new binary (0/1) variables*:

In [0]:
from sklearn.preprocessing import OneHotEncoder
hotEncode = OneHotEncoder()
Encoded = pd.DataFrame(hotEncode.fit_transform(proximity).toarray())
Encoded.head()

In [0]:
proximity.join(Encoded).sample(10)

You could then pipe this in to your model to get more explanation for housing prices, etc.

## Training Sets, Test Sets, and Model Evaluation

In practice, a *very* important preprocessing step is to split up your data into a *training set* and a *testing set*.

In [0]:
from sklearn.model_selection import train_test_split
trainingData, testingData = train_test_split(housing, test_size = .2, random_state = 3141592653)

In [0]:
trainingData

In [0]:
testingData

Once you do this, in practice you *completely forget about the test set* until you are done building your "model". Then the test set is used to, well, TEST your model. This is done in part to help prevent overfitting.

In [0]:
ols.fit(trainingData[['housing_median_age','median_income']], trainingData['median_house_value'])

In [0]:
predictions = ols.predict(testingData[['housing_median_age','median_income']])
predictions

In [0]:
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions, testingData['median_house_value'])**(1/2)

This is a *very large error*, signifying that the linear model is *underfitting* the data. Even if we add more explanatory variables, we still get a quite poor error:

In [0]:
ols.fit(trainingData[['housing_median_age','median_income', 'total_bedrooms']], trainingData['median_house_value'])

In [0]:
predictions = ols.predict(testingData[['housing_median_age','median_income','total_bedrooms']])
predictions

In [0]:
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions, testingData['median_house_value'])**(1/2)

This is simply because linear models are too crude of a model for this data. Perhaps if we have extra time at the end of the quarter we will discuss more powerful techniques for studying this data set.

I think this is all for today. Please note; you could spend *months* practicing building your pipelines and working with pandas. There is much more we didn't get to here, such as 
- How to construct dataframes "from scratch" in Python
- SQL style table operations
- Pivot tables
- etc...